In [ ]:
import pandas as pd
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
import requests
import datetime
import json

# Wczytanie danych GTFS-RT
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://gtfs.ztp.krakow.pl/VehiclePositions_A.pb')
feed.ParseFromString(response.content)


entity_dict = MessageToDict(feed.entity[0], preserving_proto_field_name=True)

# Ładne wyświetlenie JSON
print(json.dumps(entity_dict, indent=2))
# Lista na dane
vehicles = []

# Przetwarzanie danych
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle = entity.vehicle

        vehicles.append({
            'vehicle_id': vehicle.vehicle.id,
            'trip_id': vehicle.trip.trip_id,
            'route_id': vehicle.trip.route_id,
            'latitude': vehicle.position.latitude,
            'longitude': vehicle.position.longitude,
            'bearing': vehicle.position.bearing,
            'speed': vehicle.position.speed,
            'timestamp': datetime.datetime.fromtimestamp(vehicle.timestamp),
            'stop_id':vehicle.stop_id,
            'current_stop_sequence':vehicle.current_stop_sequence,
            'status': vehicle.vehicle_stop_status,
            'current_date': datetime.date.today(),
            'day': datetime.date.today().isoweekday()
        })

# Konwersja do DataFrame
df = pd.DataFrame(vehicles)

# Zapis do CSV (opcjonalnie)
df.to_csv('vehicle_positions.csv', index=False)

# Podgląd
print(df.head())

In [2]:
stop_times=pd.read_csv('../dane/stop_times.txt',delimiter=',')
stop_times.head()
stop_times.drop(columns='stop_headsign')


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,block_1_trip_2_service_1,05:03:00,05:03:00,stop_2961_372404,1,0,1,0.000,1
1,block_1_trip_2_service_1,05:04:00,05:04:00,stop_1082_260101,2,0,0,0.610,1
2,block_1_trip_2_service_1,05:05:00,05:05:00,stop_42_6001,3,0,0,0.870,1
3,block_1_trip_2_service_1,05:08:00,05:08:00,stop_1535_306902,4,0,0,1.910,1
4,block_1_trip_2_service_1,05:09:00,05:09:00,stop_65_9001,5,0,0,2.420,1
...,...,...,...,...,...,...,...,...,...
883218,block_2128_trip_13_service_5,19:19:00,19:19:00,stop_50_7104,16,0,0,13.420,1
883219,block_2128_trip_13_service_5,19:22:00,19:22:00,stop_55_7904,17,0,0,14.320,1
883220,block_2128_trip_13_service_5,19:24:00,19:24:00,stop_1626_311102,18,0,0,15.015,1
883221,block_2128_trip_13_service_5,19:26:00,19:26:00,stop_1654_314104,19,0,0,15.495,1


In [12]:
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
import requests
import json

# Wczytanie danych GTFS-RT
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://gtfs.ztp.krakow.pl/TripUpdates_A.pb')
feed.ParseFromString(response.content)

entity_dict = MessageToDict(feed.entity[0], preserving_proto_field_name=True)

# Ładne wyświetlenie JSON
print(json.dumps(entity_dict, indent=2))

# Lista na dane
vehicles = []

# Przetwarzanie danych
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle = entity.vehicle

        vehicles.append({
            'trip_id': vehicle.trip_update.id,
            ''
        })

# Konwersja do DataFrame
df = pd.DataFrame(vehicles)

# Zapis do CSV (opcjonalnie)
df.to_csv('vehicle_positions.csv', index=False)

# Podgląd
print(df.head())

{
  "id": "block_974_trip_17",
  "trip_update": {
    "trip": {
      "trip_id": "block_974_trip_17_service_3"
    },
    "stop_time_update": [
      {
        "departure": {
          "time": "1745777400"
        },
        "stop_id": "stop_36_5401"
      },
      {
        "arrival": {
          "time": "1745777520"
        },
        "departure": {
          "time": "1745777520"
        },
        "stop_id": "stop_580_82202"
      },
      {
        "arrival": {
          "time": "1745777640"
        },
        "departure": {
          "time": "1745777640"
        },
        "stop_id": "stop_7_1203"
      },
      {
        "arrival": {
          "time": "1745777700"
        },
        "departure": {
          "time": "1745777700"
        },
        "stop_id": "stop_2309_345907"
      },
      {
        "arrival": {
          "time": "1745777760"
        },
        "departure": {
          "time": "1745777760"
        },
        "stop_id": "stop_105_13701"
      },
      {
        "